In [1]:
#CLSP-Rmin with back-ordering example
from gurobipy import *
import xlsxwriter
import time
import math
import random
from function_call import compute_values
from function_call_clsp import compute_values as cpCLSP 
from data import variables
import json


#start = time.time()

n = Model("CLSP-RminBO")

#Parameters
d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe = variables()

# generate random ratio for defective items
random.seed(123)
o1 = round(random.uniform(0.01,0.02), 3)# realistic
o2 = round(random.uniform(0.02,0.03), 3) # pessimistic 
o3 = round(random.uniform(0.001,0.01), 3) # optimistic 
O=[o1 for t in Pe] # realistic
O1=[o2 for t in Pe ]  # pessimistic
O2=[o3 for t in Pe] # optimistic 

realizationO= [round(random.uniform(0.01,0.02),3) for t in Pe]
realizationO1= [round(random.uniform(0.02,0.03),3) for t in Pe]
realizationO2= [round(random.uniform(0.001,0.01),3) for t in Pe]

lista=[{'O':O, 'O1':O1,'O2':O2,'realizationO':realizationO,
        'realizationO1':realizationO1,'realizationO2':realizationO2}]
with open('data.json', 'w') as fa:
    json.dump(lista,fa)
json_str = json.dumps(lista)
print(json_str)
with open('data.json', 'r') as fa:
    thisfile=json.load(fa)
O=thisfile[0]["O"]
realizationO=thisfile[0]["realizationO"]


#################################### Run optymalization ############################# 


#Pe=range(T)    
#CostBO=True
#Rp=10
#cpCLSP(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)


T=6    
Pe=range(T)    
Rp=2


A1 = 0
B1 = 0
C1 = 0
D1 = 0
E1 = 0 
F1 = 0
G1 = 0
H1 = 0
I1 = 0
J1 = 0
K1 = 0

oH = 0
ohR = 0
cC = 0
oF = 0
oxprod = 0
fcostr = 0
oxrework = 0 
oCU = 0
ZZ = 0
cbcost = 0
cbcost2 = 0




for i in range(5):
    A1 = A1 + oH
    B1 = B1 + ohR
    C1 = C1 + cC
    D1 = D1 + oF
    E1 = E1 + oxprod
    F1 = F1 + fcostr
    G1 = G1 + oxrework
    H1 = H1 +  oCU
    I1 = I1 + ZZ
    J1 = J1 + cbcost
    K1 = K1 + cbcost2
    if i ==0:
        print('step',i+1)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
    
    elif i==3: #and T >len(b) and Rp<len(b):
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print(BO0)
        print(s0)
        print(sR0)
        print(b)
        print(d)
        Pe=range(len(b))
        T=len(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
    
    elif i==4: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        print(BO0)
        print(s0)
        print(sR0)
        print(b)
        print(d)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)
    
    elif i<3:
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print(i)
        print(len(b))
        print(b)
        [BO0,s0,sR0,
         oH, ohR, cC, 
         oF, oxprod, fcostr, 
         oxrework, oCU, ZZ, cbcost, cbcost2] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,realizationO)

    else:
        print('end')

workbook = xlsxwriter.Workbook('out.xlsx')
# The workbook object is then used to add new
# worksheet via the add_worksheet() method.
worksheet = workbook.add_worksheet()
 
# Use the worksheet object to write
# data via the write() method.
worksheet.write('A1', "Table 4 h Holding cost")
worksheet.write('B1', "Table 4 hR Holding cost for rework")
worksheet.write('C1', "Setup cost combined")
worksheet.write('D1', "Table 4 f Setup cost for production")
worksheet.write('E1', "Table 4 x Nr of set-ups for production")
worksheet.write('F1', "Table 4 f Setup cost for rework")
worksheet.write('G1', "Table 4 xR Setup cost for rework")
worksheet.write('H1', "Table 4 Capacity Utilization")
worksheet.write('I1', "Table 4 Z objective function value")
worksheet.write('J1', "Table 4 Back order costs cb")
worksheet.write('K1', "Table 4 Back order realization costs cb")
worksheet.write('A2',A1)
worksheet.write('B2',B1)
worksheet.write('C2',C1)
worksheet.write('D2',D1)
worksheet.write('E2',E1)
worksheet.write('F2',F1)
worksheet.write('G2',G1)
worksheet.write('H2',H1/5)
worksheet.write('I2',I1)
worksheet.write('J2',J1)
worksheet.write('K2',K1)
# Finally, close the Excel file
# via the close() method.
workbook.close()        
        


Restricted license - for non-production use only - expires 2024-10-28
[{"O": [0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011], "O1": [0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021], "O2": [0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005], "realizationO": [0.011, 0.019, 0.01, 0.015, 0.013, 0.019, 0.012, 0.013, 0.013, 0.012], "realizationO1": [0.02, 0.024, 0.021, 0.026, 0.021, 0.023, 0.024, 0.029, 0.021, 0.021], "realizationO2": [0.008, 0.001, 0.009, 0.006, 0.003, 0.009, 0.008, 0.004, 0.008, 0.003]}]
step 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 350 rows, 216 columns and 860 nonzeros
Model fingerprint: 0xc567a525
Variable types: 72 continuous, 144 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+03]

MIP start from previous solve produced solution with objective 5594.25 (0.16s)
MIP start from previous solve produced solution with objective 5594.25 (0.31s)
Loaded MIP start from previous solve with objective 5594.25

Presolve removed 506 rows and 232 columns
Presolve time: 0.01s
Presolved: 194 rows, 200 columns, 648 nonzeros
Variable types: 40 continuous, 160 integer (48 binary)

Root relaxation: objective 2.159003e+03, 156 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2159.00339    0  120 5594.25000 2159.00339  61.4%     -    0s
     0     0 4917.13110    0  137 5594.25000 4917.13110  12.1%     -    0s
H    0     0                    5590.5000000 4925.59445  11.9%     -    0s
     0     0 4925.59445    0  133 5590.50000 4925.59445  11.9%     -    0s
     0     0 5100.76215    0  115 5590.50000 5100.76215  8.76%     -    0s


     0     0 728952.304    0  116 775350.000 728952.304  5.98%     -    0s
     0     0 729034.737    0  123 775350.000 729034.737  5.97%     -    0s
     0     0 729044.621    0  130 775350.000 729044.621  5.97%     -    0s
     0     0 730261.307    0  121 775350.000 730261.307  5.82%     -    0s
     0     0 730447.162    0  124 775350.000 730447.162  5.79%     -    0s
     0     0 730470.208    0  117 775350.000 730470.208  5.79%     -    0s
     0     0 730930.850    0  114 775350.000 730930.850  5.73%     -    0s
     0     0 730946.762    0  131 775350.000 730946.762  5.73%     -    0s
     0     0 731083.392    0  121 775350.000 731083.392  5.71%     -    0s
     0     0 731083.392    0  121 775350.000 731083.392  5.71%     -    0s
     0     2 731083.392    0  121 775350.000 731083.392  5.71%     -    0s
H  554   308                    771484.25000 733805.642  4.88%  15.2    0s
H 2293   918                    769825.35768 737832.550  4.16%  15.7    2s
H 2298   874             

     0     0 774335.972    0   94 828660.000 774335.972  6.56%     -    1s
     0     0 774458.285    0   92 828660.000 774458.285  6.54%     -    1s
     0     0 774521.865    0   95 828660.000 774521.865  6.53%     -    1s
     0     0 774524.810    0   94 828660.000 774524.810  6.53%     -    1s
     0     0 774896.021    0   92 828660.000 774896.021  6.49%     -    1s
     0     0 775211.768    0   97 828660.000 775211.768  6.45%     -    1s
     0     0 775236.777    0   95 828660.000 775236.777  6.45%     -    1s
     0     0 775281.178    0  100 828660.000 775281.178  6.44%     -    1s
     0     0 775281.178    0  100 828660.000 775281.178  6.44%     -    1s
H  589   275                    819827.00000 775683.623  5.38%  10.2    1s
H  673    73                    813470.50000 776870.953  4.50%   2.8    1s
H  772   118                    813046.50000 779748.623  4.10%   3.6    1s
H 1065   239                    811434.50000 788390.042  2.84%   5.0    2s
H 1167   262             

     0     0 860838.508    0   33 865741.000 860838.508  0.57%     -    1s
     0     0 860839.044    0   33 865741.000 860839.044  0.57%     -    1s
     0     0 860839.221    0   33 865741.000 860839.221  0.57%     -    1s
H    0     0                    865711.00000 860839.221  0.56%     -    1s
     0     0 860883.273    0   30 865711.000 860883.273  0.56%     -    1s
     0     0 861019.950    0   30 865711.000 861019.950  0.54%     -    1s
     0     2 861019.950    0   30 865711.000 861019.950  0.54%     -    1s
 15218   851 865412.087   42   29 865711.000 863406.464  0.27%   3.7    7s

Explored 15218 nodes (56633 simplex iterations) in 7.34 seconds (1.54 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 865711 865741 865741 ... 870131

Optimal solution found (tolerance 1.00e-04)
Best objective 8.657110000000e+05, best bound 8.657110000000e+05, gap 0.0000%

    Variable            x 
-------------------------
     yR[0,4]            8 
     yR[1,4]  